In [3]:
import time
import os
from collections import defaultdict
import sqlite3
import shutil
import statistics
import math
import json
import string

UPDATED_ENGLISH_FILE = 'English_Updated.txt'
UPDATED_DUTCH_FILE = 'Dutch_Updated.txt'

TRAINING_ENG_FILE = 'eng_file2.txt'
TRAINING_DUTCH_FILE = 'dutch_file2.txt'

FILE_TO_BE_TESTED = 'testDoc.txt'
REWRITE_FILE = 'translated_doc.txt'

tr_etof = {}
tr_ftoe = {}

con = sqlite3.connect('TransProb_100.db')
crsr = con.cursor()

eng_data = []
dutch_data = []

eng_cleaned = []
dut_cleaned = []

translated_doc = []
test_file_lines = []

total_no_of_sentences = -1

dutchWords_line_no = defaultdict(set)
engWords_line_no = defaultdict(set)


num_dict_eng = {}
num_dict_dutch = {}

total = {}


In [4]:
def EngToDutchStopWords():
    eng_sw=['of', 'the', 'I', 'on', 'and', 'would', 'to', 'you', 'a', 'in', 'that', 'as', 'have', 'for', 'be', 'from', 'it', 'at', 'can', 'an', 'has', 'The', 'It', 'is', 'not', 'with', 'We', 'by', 'This', 'we', 'are', 'more', 'our', 'or', 'also', 'these', 'but', 'must']
    dut_sw=['van', 'de', 'ik', 'Aan', 'en', 'Zou', 'naar', 'u', 'een', 'in', 'dat', 'als', 'hebben', 'voor', 'worden', 'van', 'het', 'Bij', 'kan', 'een', 'heeft', 'De', 'Het', 'is', 'niet', 'met', 'Wij', 'door', 'Deze', 'wij', 'zijn', 'meer', 'onze', 'of', 'ook', 'deze', 'maar', 'moet']
    eng_sw = [i.lower() for i in eng_sw]
    dut_sw = [i.lower() for i in dut_sw]
    
    return (dict(zip(eng_sw, dut_sw)))


In [5]:
def DutchToEngStopWords():
    eng_sw=['of', 'the', 'I', 'on', 'and', 'would', 'to', 'you', 'a', 'in', 'that', 'as', 'have', 'for', 'be', 'from', 'it', 'at', 'can', 'an', 'has', 'The', 'It', 'is', 'not', 'with', 'We', 'by', 'This', 'we', 'are', 'more', 'our', 'or', 'also', 'these', 'but', 'must']
    dut_sw=['van', 'de', 'ik', 'Aan', 'en', 'Zou', 'naar', 'u', 'een', 'in', 'dat', 'als', 'hebben', 'voor', 'worden', 'van', 'het', 'Bij', 'kan', 'een', 'heeft', 'De', 'Het', 'is', 'niet', 'met', 'Wij', 'door', 'Deze', 'wij', 'zijn', 'meer', 'onze', 'of', 'ook', 'deze', 'maar', 'moet']
    eng_sw = [i.lower() for i in eng_sw]
    dut_sw = [i.lower() for i in dut_sw]
    
    return (dict(zip( dut_sw,eng_sw)))

In [6]:
def readfiles(Eng_File, Dutch_File,no_of_sentences):
    """
    Function to read required number of lines(no_of_sentences) from given text files  
    
    """
    e_file = open(Eng_File,'r',encoding = 'utf-8')
    d_file = open(Dutch_File,'r',encoding = 'utf-8')
    
    global dutch_data
    global eng_data
    dutch_data = d_file.readlines()[:no_of_sentences]
    eng_data = e_file.readlines()[:no_of_sentences]
    

In [7]:
def remove_punc(l,lang):
    """
    Function to remove punctuation symbols from acquired sentences

    """
    
    for i in range(len(l)):
        l[i] = l[i][:-1].lower()
        l[i] = remove_stopwords(l[i],lang)
        l[i] = remove_stuff(l[i])
    return l

#CHECKED
def remove_stopwords(l,lang):
    eng_sw=['of', 'the', 'i', 'on', 'and', 'would', 'to', 'you', 'a', 'in', 'that', 'as', 'have', 'for', 'be', 'from', 'it', 'at', 'can', 'an', 'has', 'the', 'it', 'is', 'not', 'with', 'we', 'by', 'this', 'we', 'are', 'more', 'our', 'or', 'also', 'these', 'but', 'must']
    dut_sw=['van', 'de', 'ik', 'aan', 'en', 'zou', 'naar', 'u', 'een', 'in', 'dat', 'als', 'hebben', 'voor', 'worden', 'van', 'het', 'bij', 'kan', 'een', 'heeft', 'de', 'het', 'is', 'niet', 'met', 'wij', 'door', 'deze', 'wij', 'zijn', 'meer', 'onze', 'of', 'ook', 'deze', 'maar', 'moet']
    eng_sw = [i.lower() for i in eng_sw]
    dut_sw = [i.lower() for i in dut_sw]
    
    words = l.split()
    s = []
    
    if lang =='dutch':
        for i in words:
            if i not in dut_sw:
                s.append(i)
    elif lang =='eng':
        for i in words:
            if i not in eng_sw:
                s.append(i)
    return ' '.join(s)

#CHECKED    
def remove_stuff(l):
    #a = [ ':','.' , '\\' , '/' , ',' , ';' , '(' , ')' , '"', "\'",'1','2','3','4','5','6','7','8','9','0','?']
    for i in string.punctuation + string.digits:
        l = l.replace(i, "")
    return l


In [8]:
def assign_line_no(doc):
    dict_lo = defaultdict(set)
    for i in range(len(doc)):
        t = doc[i].split()
        for m in t:
            dict_lo[m].add(i)
    return dict_lo

In [9]:
def finding_probabilities():
    """
    Function to update count for each english-dutch pair
    
    """

    global total
    global eng_cleaned
    global dut_cleaned
    global total_no_of_sentences


    for i in range(total_no_of_sentences):
        
        
        en = eng_cleaned[i]
        en_words = en.split()
        

        du = dut_cleaned[i]
        du_words = du.split()
        

        retrieved_count = {}
        retrieved_term_probability = {}

        for e in en_words:

            for d in du_words:

                comm = "select * from TransProb where ENG_WORD= '{0}' AND DUT_WORD= '{1}'".format(e,d)
                com2 = "select * from Count where ENG_WORD= '{0}' AND DUT_WORD= '{1}'".format(e,d)
                #print(comm)
                #print(com2)
                
                
                crsr.execute(comm)
                ans = crsr.fetchall()
                
                crsr.execute(com2)
                an2 = crsr.fetchall()
                try:
                    line_no2= ans[0][0]+'_'+ans[0][1]+' '+str(ans[0][2])
                    line_no = an2[0][0]+'_'+an2[0][1]+' '+str(an2[0][2])
                except:
                    print(line_no2)
                pr = line_no.split()
                pr2 = line_no2.split()
                
                #print(pr)
                #print(pr2)
                retrieved_count[pr[0]] = float(pr[1])
                retrieved_term_probability[pr2[0]] = float(pr2[1])
              

        #print("Count and Translation probabilities retrieved")
        
        #print(retrieved_term_probability)
        #print()

        s_total = {}

        for e in en_words:

            s_total[e] = 0

            for d in du_words:

                s = e+'_'+d

                s_total[e] += retrieved_term_probability[s]

       #print(s_total)
       # print("S_total for each english word done")
                
        for e in en_words:

            for d in du_words:

                s = e+'_'+d

                retrieved_count[s] += (retrieved_term_probability[s]/s_total[e])

                total[d] += (retrieved_term_probability[s]/s_total[e])

        #print("Counts modified and Total calculated")
        #print(retrieved_count)
        #print()
        #print("RETRIEVED TP")
        #print(retrieved_term_probability)
        #print()
        
        for k,v in retrieved_count.items():
            
            t = k.split('_')
            e = t[0]
            d = t[1]
            #print(k)
            #print(e+" "+d)
            
            try:
                command= "UPDATE Count SET PROBABILITY={0} WHERE ENG_WORD = '{1}' AND DUT_WORD ='{2}' ".format(v,e,d)
                #print(command)
            
                crsr.execute(command)
            except:
                pass
            
                
        con.commit()
        
        #print("Writeback completed into count file")

In [10]:
def running_function(no_of_iterations=7):
    """
    Function to reinitialize total for foreign words and count for eng-dutch pairs. It carries out (no_of_iterations) iterations
    It rewrites the translational probabilities back into the CondProb file.
    """
    
    global total
    global dutchWords_line_no
    global engWords_line_no
    global total_no_of_sentences
    global dut_cleaned
    global eng_cleaned



    for c_i in range(no_of_iterations):
        
        print(c_i+1)
        
        for i in engWords_line_no.keys():
            for j in dutchWords_line_no.keys():
                command= "UPDATE Count SET PROBABILITY= {0} WHERE ENG_WORD = '{1}' AND DUT_WORD ='{2}' ".format(0,i,j)
                #print(command)
                crsr.execute(command)
            con.commit()
            
                
    
        total = {}
    
        for k in dutchWords_line_no.keys():
            total[k] = 0
    
        finding_probabilities()
    
    
        print("Finding probabilities done, ",c_i)
        retrieved_count = {}
        retrieved_term_probability = {}
    
       # print(total)
        
        
        for e in engWords_line_no.keys():
            
            for d in dutchWords_line_no.keys():
                
                com2 = "select * from Count where ENG_WORD= '{0}' AND DUT_WORD= '{1}'".format(e,d)
                #print(com2)
                crsr.execute(com2)
                ans = crsr.fetchall()
                
                try:
                    line_no = ans[0][0]+'_'+ans[0][1]+' '+str(ans[0][2])
                except:
                    print(line_no)
                
                pr = line_no.split()
                
                retrieved_count[pr[0]] = float(pr[1])
            con.commit()
        
    

        print("Counts, retrieved")
        
        print()
        #print(retrieved_count)
        print()
    
        for d in dutchWords_line_no.keys():
            for e in engWords_line_no.keys():
            
                s = e+ '_' +d
                try:
                    retrieved_term_probability[s] = retrieved_count[s]/total[d]  
                except:
                    print(d);
                
                try:
                    command= "UPDATE TransProb SET PROBABILITY= {0} WHERE ENG_WORD = '{1}' AND DUT_WORD ='{2}' ".format(float(retrieved_term_probability[s]),e,d)
                    #print(command)
                
                    crsr.execute(command)
                except:
                    pass
                
            con.commit()
        
        print("Translational Probabilites updated")
        

In [11]:
def retrieve_max(english_l):
    """
    Function to retieve the translaton with maximum probabilities for each word to be translated, i.e. to obtain the most probable translation for each word.     
    """
    global tr_etof
    global tr_ftoe
    for e in english_l.keys():
        
        #print(e,counter)
        
        comm = "SELECT MAX(PROBABILITY),DUT_WORD From TransProb where ENG_WORD = '{0}'".format(e)
        
        crsr.execute(comm)
        
        ans = crsr.fetchall()
         
        tr_etof[e] = ans[0][1]
        tr_ftoe[ans[0][1]] = e

In [12]:
def remove_punctuation(l):
    """
    Function to remove punctuation symbols from acquired sentences

    """
    
    for i in range(len(l)):
        l[i] = l[i][:-1].lower()
        l[i] = remove_stuff(l[i])
    return l

In [13]:
def pearson_coefficient():
    """
    Function to find Pearson Coefficient
    """
    
    # Make sure these lists do not have anything other than words
    global translated_doc
    global test_file_lines    
    
    translated_doc_line_no = assign_line_no(translated_doc)
    test_file_lines_line_no = assign_line_no(test_file_lines)

    total_sim = 0

    tf_of_cleaned = maintain_count(test_file_lines_line_no,test_file_lines)
    
    tf_of_result = maintain_count(translated_doc_line_no,translated_doc)
    
    avg_cleaned = statistics.mean(tf_of_cleaned.values())
    avg_result = statistics.mean(tf_of_result.values())
    
    for i in tf_of_cleaned.keys():
        try:
            total_sim += ((tf_of_cleaned[i]-avg_cleaned) * (tf_of_result[i] - avg_result))
        except:
            pass
        
    stddev_cleaned = statistics.stdev(tf_of_cleaned.values())
    stddev_result = statistics.stdev(tf_of_result.values())
    
    return total_sim/(stddev_cleaned*stddev_result*len(tf_of_cleaned))

In [14]:
def maintain_count(doc_dict_line_no,data_lines):
    
    og_dutch_tf = {}
    for dword in doc_dict_line_no:                #for each word in the document
        dlist=[]
        dlist=list(doc_dict_line_no[dword])
        count = 0
        for i in dlist:
            count+= data_lines[i].count(dword)
        
        og_dutch_tf[dword] = (count)

    return og_dutch_tf

In [15]:
def cosine_similarity():
    """
    Function to find cosine similarity between 2 documents
    
    Required: Clean test data before calculating cosine_similarity
    """
    global translated_doc
    global test_file_lines    
    
    translated_doc_line_no = assign_line_no(translated_doc)
    test_file_lines_line_no = assign_line_no(test_file_lines)
    
    tf_of_cleaned = maintain_normalized_tf(test_file_lines_line_no,test_file_lines)
    
    tf_of_result = maintain_normalized_tf(translated_doc_line_no,translated_doc)
    
    total_sim = 0
    
    for i in tf_of_cleaned.keys():
        try:
            total_sim += tf_of_cleaned[i] * tf_of_result[i]
        except:
            pass
        
    return total_sim

In [16]:
def maintain_normalized_tf(doc_dict_line_no,data_lines):
    
    """
    Function to find normalized term-frequency of a word in a document.
    
    """
    og_dutch_tf = {}
    
    sum_of_tf = 0
    
    for dword in doc_dict_line_no:                #for each word in the document
        dlist=[]
        dlist=list(doc_dict_line_no[dword])
        count = 0
        for i in dlist:
            count+= data_lines[i].count(dword)
        
        og_dutch_tf[dword] = math.log(count)+1
        
        sum_of_tf += pow(og_dutch_tf[dword],2)
    
    normalized_denom = pow(sum_of_tf,0.5)
    
    for i in og_dutch_tf.keys():
        og_dutch_tf[i] = og_dutch_tf[i]/normalized_denom

        
    return og_dutch_tf

In [17]:
def produce_sentence(eng_sentence):
    """
    Function to translate a sentence.
    """
    
    s = ""

    global tr_etof
    global tr_ftoe
    
    temp_copy = eng_sentence
    
    new_sen = temp_copy[:-1].lower()
    new_sen = remove_stuff(new_sen)
    
    for i in new_sen.split():
         
        try:
            s += str(tr_etof[i])
        except:
            s += str(i)
        s += " "
    
    s+='\n'
    
    return s

In [18]:
def translate_doc(list_lines_to_be_translated):
    """
    Function to translate a document.
    """
    global translated_doc
    
    translated_doc = []
    
    for i in list_lines_to_be_translated:
        s = produce_sentence(i)
        translated_doc.append(s)
    
    with open(REWRITE_FILE,'w') as f:
        f.writelines(translated_doc)
    

In [19]:
def main():
    
    
    Eng_File = TRAINING_ENG_FILE
    Dutch_File = TRAINING_DUTCH_FILE
    

    readfiles(Eng_File,Dutch_File,10000)
    
    print("Files read")
    
    # Cleaning Data

    global eng_cleaned
    global dut_cleaned
    
    eng_cleaned = remove_punc(eng_data,'eng')
    dut_cleaned = remove_punc(dutch_data,'dutch')

    global total_no_of_sentences
    
    total_no_of_sentences = len(dut_cleaned)

    assert(total_no_of_sentences!=-1)
    
    print("Files cleaned")

    global dutchWords_line_no
    global engWords_line_no
    
    dutchWords_line_no = assign_line_no(dut_cleaned)
    engWords_line_no = assign_line_no(eng_cleaned)
    
    no_of_words_eng = len(engWords_line_no)
    no_of_words_dutch = len(dutchWords_line_no)
    
    print("Initialization Done")
    
    running_function(7)
    
    print("Model Trained")
    
    global tr_etof
    global tr_ftoe

    retrieve_max(engWords_line_no)
    
    tr_etof.update(EngToDutchStopWords())
    tr_ftoe.update(DutchToEngStopWords())
    
    print(tr_etof)
    print("\n\n\n\n")
    print(tr_ftoe)
    
    with open('etof.json','w') as fp:
        json.dump(tr_etof,fp)
    
    with open('ftoe.json','w') as fp:
        json.dump(tr_ftoe,fp)
    
    #new_doc = open(TEST_FILE,'r+')
    #lines = new_doc.readlines()
    
    #b = translate_doc(lines,tr_etof,TEST_FILE)
    
    #print(b)
    

In [20]:
if __name__ == "__main__":
    main()

Files read
Files cleaned
Initialization Done
1
Finding probabilities done,  0
Counts, retrieved


inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
in

inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inger
inge

uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitn

uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitn

verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwa

...

uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitn

uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitnodiging
uitn

verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwa

verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwaardigde
verontwa

Translational Probabilites updated
Model Trained
{'resumption': 'hervatting', 'session': 'zitting', 'agenda': 'werkzaamheden', 'relating': 'februari', 'wednesday': 'woensdag', 'applause': 'applaus', 'pse': 'pse', 'group': 'fractie', 'thank': 'dank', 'you': 'u', 'mr': 'heer', 'poettering': 'poettering', 'lot': 'krediet', 'ask': 'honoreren', 'there': 'er', 'no': 'geen', 'room': 'scherper', 'amendments': 'amendementen', 'did': 'gaf', 'happen': 'veeleer', 'important': 'herkenbaarheid', 'matter': 'belangrijkst', 'call': 'biosciencenorth', 'me': 'mij', 'either': 'either', 'seems': 'lijkt', 'workable': 'oplossing', 'solution': 'oplossing', 'pity': 'zekere', 'sense': 'consensus', 'short': 'kortom', 'issue': 'instantie', 'one': 'één', 'work': 'samenwerken', 'done': 'geschiedt', 'all': 'allemaal', 'remains': 'onbeantwoord', 'business': 'inspecties', 'enforcement': 'naleving', 'like': 'wil', 'mention': 'onvermeld', 'final': 'smeekbede', 'point': 'punt', 'this': 'deze', 'however': 'echter', 'does'

In [ ]:
import statistics
import math

In [ ]:
def Test():
    global translated_doc
    global tr_etof
    global tr_ftoe
    global test_file_lines
    
    
    TEST_ENG = 'test_eng.txt'
    TEST_DUT = 'test_dutch.txt'
    
    test_e = open(TEST_ENG,'r+')
    test_d = open(TEST_DUT,'r+')
    
    eng_test = test_e.readlines()
    dut_test = test_d.readlines()
    
    with open('etof.json','r+') as fp:
        tr_etof= json.load(fp)
    with open('ftoe.json','r+') as fp:
        tr_ftoe= json.load(fp)
    

    translate_doc(eng_test)
    
    test_file_lines = dut_test
    
    translated_doc = remove_punctuation(translated_doc)
    test_file_lines = remove_punctuation(test_file_lines)

    print(pearson_coefficient())
    print(cosine_similarity())

In [ ]:
Test()

In [ ]:
con.close()